In [1]:
# Capstone A Project - Spring 2020
# Kaggle Attrition data - Exploratory Data Analysis Python Code 
# Reference URL:  https://www.kaggle.com/abhishektibrewal/hr-employee-attrition

# data transformation
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime

# data plotting
import seaborn as sns
#from matplotlib import pyplot as plt
import matplotlib.pyplot as plt

# classification & correlation
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics as mt
#import xgboost as xgb




C:\Users\shayden\Anaconda3\envs\RPy\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if __name__ == '__main__':


In [2]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf



print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
## Importing data from an Excel file:
## Prior to loading the file, convert it to csv in Excel 
## Setting datatypes prior to load

## updated data types to match types in IBM's as listed here:  
## http://inseaddataanalytics.github.io/INSEADAnalytics/groupprojects/January2018FBL/IBM_Attrition_VSS.html

datatypes = {
"Age": "int",
"Attrition": "category",
"BusinessTravel": "category",
"DailyRate": "int",
"Department": "category",
"DistanceFromHome": "int",
"Education": "int", ## changed to int
"EducationField": "category",
"EmployeeCount": "int",
"EmployeeNumber": "int",
"EnvironmentSatisfaction": "int", ## changed
"Gender": "category",
"HourlyRate": "int",
"JobInvolvement": "int", ## changed
"JobLevel": "int", #changed
"JobRole": "category",
"JobSatisfaction": "int", ## changed
"MaritalStatus": "category",
"MonthlyIncome": "int",
"MonthlyRate": "int",
"NumCompaniesWorked": "int",
"Over18": "category",
"OverTime": "category",
"PercentSalaryHike": "float",
"PerformanceRating": "int", ## changed
"RelationshipSatisfaction": "int", ## changed
"StandardHours": "int",
"StockOptionLevel": "int",  ## changed
"TotalWorkingYears": "int",
"TrainingTimesLastYear": "int",
"WorkLifeBalance": "int", #changed
"YearsAtCompany": "int",
"YearsInCurrentRole": "int",
"YearsSinceLastPromotion": "int",
"YearsWithCurrManager": "int"
}

df = pd.read_csv('https://raw.githubusercontent.com/skhayden/SMU-Capstone-Age-Bias-in-Predictive-Modeling-/master/WA_Fn-UseC_-HR-Employee-Attrition.csv',low_memory=False)


# Verifying the upload of the data looking at the first 5 rows
df.head()


In [ ]:
# Verifying the upload of the data looking at the last 5 rows

df.tail()


In [ ]:
# Verifying the dimensions of the data set - (rows, cols)

df.shape

In [ ]:
# Check what data types the features are

df.info()

In [ ]:
df.isnull().sum()


In [ ]:
# Retrieve simple statistics from the data set

df.describe()


In [ ]:
# Exploring our Age feature

df.Age.unique()

In [ ]:
# Exploring our Age feature

df.Age.value_counts()

In [ ]:
#decade increments
bin_labels10 = ['<=30', '>30 & <=40', '>40 & <=50', '>50']
bins10 = [min(df['Age']), 30, 40, 50, max(df['Age'])]
df['Age_group_Decade'] = pd.cut(df['Age'], bins=bins10, labels=bin_labels10)


#half decade increments
bin_labels5 = ['<=25', '>25 & <=30','>30 & <=35','>35 & <=40','>40 & <=45','>45 & <=50','>50']
bins5 = [min(df['Age']), 25, 30, 35, 40, 45, 50, max(df['Age'])]
df['Age_group_half_Decade'] = pd.cut(df['Age'], bins=bins5, labels=bin_labels5)

#year increments
bin_labels1 = list(range(min(df['Age']),max(df['Age'])))
bins1 = list(range(min(df['Age']),max(df['Age'])))
df['Age_group_Year'] = pd.cut(df['Age'], bins=bins1, labels=bin_labels1[0:len(bin_labels1)-1])



fig, ((ax0, ax1,ax2)) = plt.subplots( ncols=3,figsize=(20, 10))

ax0.hist([df['Age'].loc[df['Attrition']=='Yes'],
          df['Age']],bins=bins5, stacked=True,color=['green', 'orange'], 
          rwidth=0.98,label=['Attrition Yes', 'Attrition No'])
ax0.set_title('5 Year Increments')
ax0.legend(prop={'size': 10})

ax1.hist([df['Age'].loc[df['Attrition']=='Yes'],
          df['Age']],bins=bins10, stacked=True,color=['green', 'orange'], 
          rwidth=0.98,label=['Attrition Yes', 'Attrition No'])
ax1.set_title('10 Year Increments')
ax1.legend(prop={'size': 10})

ax2.hist([df['Age'].loc[df['Attrition']=='Yes'],
          df['Age']],bins=bins1, stacked=True,color=['green', 'orange'], 
          rwidth=0.85,label=['Attrition Yes', 'Attrition No'])
ax2.set_title('1 Year Increments')
ax2.legend(prop={'size': 10})

fig.tight_layout()
plt.show()


In [ ]:
idx = pd.IndexSlice
#10 year perc
Agedf10=df.groupby(['Age_group_Decade', 'Attrition']).agg({'Attrition': 'count'})
Age10_pcts = Agedf10.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
#5 year perc
Agedf5=df.groupby(['Age_group_half_Decade', 'Attrition']).agg({'Attrition': 'count'})
Age5_pcts = Agedf5.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

#1 year perc
Agedf1=df.groupby(['Age_group_Year', 'Attrition']).agg({'Attrition': 'count'})
Age1_pcts = Agedf1.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))






In [ ]:

Age10_pcts.loc[idx[:, ['Yes']], idx['Attrition']].plot.bar(figsize=(5, 5),layout=(2, 2),
                title='Attrtion percentage by age group',rot=50)


In [ ]:
Age5_pcts.loc[idx[:, ['Yes']], idx['Attrition']].plot.bar(figsize=(5, 5),layout=(2, 2),
                title='Attrtion percentage by age group',rot=50)


In [ ]:
Age1_pcts.loc[idx[:, ['Yes']], idx['Attrition']].plot.bar(figsize=(10, 5),
                    layout=(2, 2),title='Attrtion percentage by age group',
                    rot=50)


In [ ]:
## In order to get familiarized with data set, we are displaying categorical frequency groupings for:
## categorical features 
## some numerical features with levels 
## target label (Y - Attrition)

##### Categorical Features   #####

## Attrition Frequency
df_byAttrition = df.groupby(['Attrition'])
print(df_byAttrition.Attrition.count())
print(" ")
print(" ")

## BusinessTravel Frequency
df_byBusinessTravel = df.groupby(['BusinessTravel'])
print(df_byBusinessTravel.BusinessTravel.count())
print(" ")
print(" ")

## Department Frequency
df_byDepartment = df.groupby(['Department'])
print(df_byDepartment.Department.count())
print(" ")
print(" ")

## Education Field Frequency Table
df_byEducationField = df.groupby(['EducationField'])
print(df_byEducationField .EducationField.count())
print(" ")
print(" ")

## Gender Frequency Table
df_byGender = df.groupby(['Gender'])
print(df_byGender.Gender.count())
print(" ")
print(" ")

## JobRole Frequency Table
df_byJobRole = df.groupby(['JobRole'])
print(df_byJobRole.JobRole.count())
print(" ")
print(" ")

## MaritalStatus  Frequency Table
df_byMaritalStatus  = df.groupby(['MaritalStatus'])
print(df_byMaritalStatus.MaritalStatus.count())
print(" ")
print(" ")

## Over18  Frequency Table
df_byOver18  = df.groupby(['Over18'])
print(df_byOver18.Over18.count())
print(" ")
print(" ")

## OverTime  Frequency Table
df_byOverTime  = df.groupby(['OverTime'])
print(df_byOverTime.OverTime.count())
print(" ")
print(" ")





#### Numerical features with levels  ####
print("Numerical features with levels")
print(" ")
print(" ")



## Education Frequency
df_byEducation = df.groupby(['Education'])
print(df_byEducation.Education.count())
print(" ")
print(" ")

## df_byEnvironmentSatisfaction Frequency Table
df_byEnvironmentSatisfaction = df.groupby(['EnvironmentSatisfaction'])
print(df_byEnvironmentSatisfaction.EnvironmentSatisfaction.count())
print(" ")
print(" ")

## JobInvolvement Frequency Table
df_byJobInvolvement = df.groupby(['JobInvolvement'])
print(df_byJobInvolvement.JobInvolvement.count())
print(" ")
print(" ")

## JobLevel Frequency Table
df_byJobLevel = df.groupby(['JobLevel'])
print(df_byJobLevel.JobLevel.count())
print(" ")
print(" ")

## JobSatisfaction Frequency Table
df_byJobSatisfaction = df.groupby(['JobSatisfaction'])
print(df_byJobSatisfaction.JobSatisfaction.count())
print(" ")
print(" ")

## PerformanceRating  Frequency Table
df_byPerformanceRating  = df.groupby(['PerformanceRating'])
print(df_byPerformanceRating.PerformanceRating.count())
print(" ")
print(" ")

## RelationshipSatisfaction  Frequency Table
df_byRelationshipSatisfaction  = df.groupby(['RelationshipSatisfaction'])
print(df_byRelationshipSatisfaction.RelationshipSatisfaction.count())
print(" ")
print(" ")

## StockOptionLevel  Frequency Table
df_byStockOptionLevel  = df.groupby(['StockOptionLevel'])
print(df_byStockOptionLevel.StockOptionLevel.count())
print(" ")
print(" ")

## WorkLifeBalance  Frequency Table
df_byWorkLifeBalance  = df.groupby(['WorkLifeBalance'])
print(df_byWorkLifeBalance.WorkLifeBalance.count())
print(" ")
print(" ")



In [ ]:
## Reducing levels in Education feature from 5(five) to 4 (four) levels

df_byEducation = df.groupby(['Education'])
print(df_byEducation.Education.count())
print(" ")
print(" ")

i = 0
while (i < df.Education.count()):
   ##print ("The Education value is:", df.Education[i])
   if df.Education[i] == 5:
     df.Education[i] = 4
   i = i + 1


df_byEducation = df.groupby(['Education'])
print(df_byEducation.Education.count())
print(" ")
print(" ")

Create the target with a binary flag. 1 indicates that they left, and 0 indicates they did not. 

In [ ]:
df['OverTime']

In [ ]:
#OneHotdf.columns
#list(df.select_dtypes(exclude = 'int64').columns)
#Categorical_Columns= list(df.select_dtypes(exclude = 'int64').columns)
#Categorical_Columns=Categorical_Columns[0:len(Categorical_Columns)-3]
#Categorical_Columns
#Categorical_Columns[0:len(Categorical_Columns)-3]

In [ ]:
####### Steven One hot encouding 

exclude=df.select_dtypes('int64').columns

#get columns to onehot encoude
Numeric_Columns=list(df.select_dtypes('int64').columns)
Categorical_Columns= list(df.select_dtypes(exclude = 'int64').columns)

               
########
#onehot
OneHotdf = pd.get_dummies(df[Categorical_Columns].drop(['Attrition',
                                'Age_group_Decade','Age_group_half_Decade',
                                'Age_group_Year'],axis=1))

#remove binary varible that is duplicated
OneHotdf = OneHotdf.drop(['OverTime_No','Gender_Male'],axis=1)

#df=df.drop(Categorical_Columns,axis=1)
df = pd.concat([df,OneHotdf], axis=1)


#create target 
df['Target_Attrition_Flag'] = np.where(df['Attrition']=='Yes', 1, 0)


In [ ]:
## Preprocessing, Label Encoding Attrition, the Target Label, and non-numerical value features to add to correlation matrix and
## AttritionOne will be used as the Target label in  XGBoster classifier


le = preprocessing.LabelEncoder()

AttritionOne       = le.fit_transform(df.Attrition)
BusinessTravelOne  = le.fit_transform(df.BusinessTravel)
DepartmentOne      = le.fit_transform(df.Department)
EducationFieldOne  = le.fit_transform(df.EducationField)
GenderOne          = le.fit_transform(df.Gender)
JobRoleOne         = le.fit_transform(df.JobRole)
MaritalStatusOne   = le.fit_transform(df.MaritalStatus)
Over18One          = le.fit_transform(df.Over18)
OverTimeOne        = le.fit_transform(df.OverTime)



df2 = pd.DataFrame({'AttritionOne':AttritionOne.tolist(),
                    'BusinessTravelOne':BusinessTravelOne.tolist(),
                    'DepartmentOne':DepartmentOne.tolist(),
                    'EducationFieldOne':EducationFieldOne.tolist(),
                    'GenderOne':GenderOne.tolist(),
                    'JobRoleOne':JobRoleOne.tolist(),
                    'MaritalStatusOne':MaritalStatusOne.tolist(),
                    'Over18One':Over18One.tolist(),
                    'OverTimeOne':OverTimeOne.tolist()
                   })

df3 = pd.concat([df, df2], axis=1)

## print(df2.AttritionOne)




In [ ]:
df2.AttritionOne

In [ ]:
df3.info()

In [ ]:
# Let's explore correlations between features

corr = df3.corr()
corr

In [ ]:
# Mapping feature correlation with a Seaborn heatmap  
# Referenced tutorial:  https://riptutorial.com/seaborn/example/31922/basic-correlation-plot

# Set background color / chart style
sns.set_style(style = 'white')

# Set up  matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Add diverging colormap
cmap = sns.diverging_palette(10, 250, as_cmap=True)

# Draw correlation plot
sns.heatmap(corr, 
        cmap=cmap, 
        square=True,
        linewidths=.5, cbar_kws={"shrink": .5}, ax=ax, 
        center=0,
        vmin=-1)

Create group attribute for age



df.loc[df['Age'] < 20, 'Age_Group'] = '<20'
df.loc[(df['Age'] >=20 ) & (df['Age'] < 30), 'Age_Group'] = '>=20 & <30'
df.loc[(df['Age'] >=30 ) & (df['Age'] < 40), 'Age_Group'] = '>=30 & <40'
df.loc[(df['Age'] >=40 ) & (df['Age'] < 50), 'Age_Group'] = '>=40 & <50'
df.loc[(df['Age'] >=50 ), 'Age_Group'] = '>=50'


#### Splitting Data and Training Model

Create training and testing data set. This is also where we included and excluded "**Age**"

In [ ]:
#drop Categrical columns that have dummie columns
df=df.drop(Categorical_Columns[:-3],axis=1)
#remove vars from trainning either for bias or target value
# This is where you remove "Age" by including in the line below.
Columns_to_drop_for_Training=['Target_Attrition_Flag','Age_group_Year', 'Age_group_Decade','Age_group_half_Decade','Age']

# Split into train/test 
X_train, X_test, y_train, y_test = train_test_split(df.drop(Columns_to_drop_for_Training,axis=1), df['Target_Attrition_Flag'], test_size=0.25)	


### Accuracy of model

In [ ]:
import tensorflow as tf
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
#tf.random.set_seed(seed_value)
# for later versions: 
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)


In [ ]:




from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from keras import optimizers




# Build the neural network
##model = Sequential()
##model.add(Dense(25, input_dim=X_train.shape[1], activation='relu')) # Hidden 1
##model.add(Dense(10, activation='relu')) # Hidden 2

## model.add(Dense(1)) # Output

##model.add(Dense(y_train.shape[1],activation='softmax')) # Output
##model.compile(loss='categorical_crossentropy', optimizer='adam')
##model.fit(X_train,y_train,verbose=2,epochs=100)
##mean_squared_error
##sparse_categorical_crossentropy

#del model

#model = Sequential()
#model.add(Dense(25, input_dim = X_train.shape[1], activation=LeakyReLU(alpha=0.05))) # Hidden 1
#model.add(Dense(10, activation='relu')) # Hidden 2
##model.add(Dense(y_train.shape[1],activation='softmax')) # Output
#model.add(Dense(1,activation='softmax')) # Output
#sgd = tf.keras.optimizers.SGD(lr=0.8, momentum=0.9, nesterov=True)        
#model.compile(loss='binary_crossentropy', optimizer=sgd)

## Steven's Changes
#model.add(Dense(1,activation='sigmoid')) # Output
#sgd = tf.keras.optimizers.SGD(lr=.0000001, decay=1e-6, momentum=0.65, nesterov=False)# Best parm so far
##sgd = tf.keras.optimizers.SGD(lr=.00000001, decay=1e-3, momentum=0.5, nesterov=False)
#model.compile(loss='binary_crossentropy', optimizer=sgd)
#model.fit(X_train,y_train, verbose=2, epochs=100, batch_size=64)




## try leakyReLu
leaky = lambda x: tf.keras.layers.LeakyReLU(alpha=0.8)(x)


model = Sequential()
model.add(Dense(25, input_dim = X_train.shape[1], activation=leaky)) # Hidden 1

model.add(Dense(10, activation=leaky)) # Hidden 2

model.add(Dense(1,activation='sigmoid')) # Output

sgd = tf.keras.optimizers.SGD(lr=.000001, decay=1e-6, momentum=0.7, nesterov=False)# Best parm so far
model.compile(loss='binary_crossentropy', optimizer=sgd)

#adds an early stop incase the model gets stuck with higher epoch counts 
#monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=3, 
                        #verbose=1, mode='auto', restore_best_weights=True)
model.fit(X_train,y_train, verbose=2, epochs=50, batch_size=64)



#### Break out age groups
We want to break out the age groups and compare their accuracy amongst each other to get an understanding of bias in the data set. 

In [ ]:
# get overall accuracy 
Overall_pred = pd.DataFrame(model.predict(X_test))
Overall_y_compare = y_test
Overall_score = metrics.accuracy_score(Overall_y_compare, Overall_pred.astype(np.int32))



In [ ]:
#add back in age group 
X_test=pd.merge(X_test, df.drop(df.columns.difference(Columns_to_drop_for_Training), 1), left_index=True, right_index=True)


In [ ]:
#funtion to get accuracy accrossed groups in grouped data
def Group_accuracy (x_test,y_test,Grouping,Atts_not_in_training,Print):
    # This function calculates accuracy of the model and can calculate accuracy 
    #across several groups in the data. This give you information need to 
    #produce a confusion matrix 
    
    #x_test = Test data of the explanatory variables  
    #y_test = Test data of the target variable 
    #Grouping = The variable that contains the grouping of the 
        #data that this function will use to break out accuracy 
    #Atts_not_in_training = Attributes not in x_test that where 
        #not in x_test for training the model. 
    
    Preds={}
    y_compare={}
    score={}
    
      
    for group in list(x_test[Grouping].dropna().unique()):
        
        #Get only the actuals based on the index of the 
        # independent variables within the specified age group  
        Preds[group]=pd.DataFrame(model.predict(x_test.loc[x_test[Grouping]==group].drop(Atts_not_in_training,axis=1)))
        
        #Get scores for each age group
        y_compare[group] = pd.DataFrame(y_test[x_test.loc[x_test[Grouping] == group].index])
        
        #Get scores for each age group
        score[group] = metrics.accuracy_score(y_compare[group], Preds[group].astype(np.int32))
    
    if Print==True:
        
        for key in score.keys():
            print("# of obs for age group "+ str(key) + " :{}".format(str(score[key])))

    return score


In [47]:
#Overall accuracy 
print("Overall accuracy of the model: {}".format(str(Overall_score))) 

#Get the predictions for only specified age groups
print('Age_group_Decade')
Decade_score=Group_accuracy (X_test,y_test,'Age_group_Decade',Columns_to_drop_for_Training,True)  
print('Age_group_half_Decade')
Half_Decade_score=Group_accuracy (X_test,y_test,'Age_group_half_Decade',Columns_to_drop_for_Training,True)
#print('Age_group_Year')
#Year_score=Group_accuracy (X_test,y_test,'Age_group_Year',Columns_to_drop_for_Training,True)




Overall accuracy of the model: 0.8586956521739131
Age_group_Decade
# of obs for age group >40 & <=50 :0.9213483146067416
# of obs for age group >30 & <=40 :0.8187919463087249
# of obs for age group <=30 :0.8426966292134831
# of obs for age group >50 :0.972972972972973
Age_group_half_Decade
# of obs for age group >45 & <=50 :0.9032258064516129
# of obs for age group >30 & <=35 :0.75
# of obs for age group <=25 :0.7619047619047619
# of obs for age group >35 & <=40 :0.9180327868852459
# of obs for age group >50 :0.972972972972973
# of obs for age group >25 & <=30 :0.8676470588235294
# of obs for age group >40 & <=45 :0.9310344827586207


In [ ]:

X_test['Age_group_Decade'].value_counts()


In [ ]:
X_test['Age_group_half_Decade'].value_counts()


In [ ]:
X_test['Age_group_Year'].value_counts()